# RIPPL SETUP

The following code will guide you through the process to setup your RIPPL
work environment.

To allow downloads for the SAR SLC's, DEMs and orbits you will need to 
create a few online accounts if you want to be able to download these.

In [1]:
import os
import sys
import logging
logging.getLogger().setLevel(logging.INFO)

rippl_path = input('Enter the full path to your RIPPL folder')

try:
    if os.path.exists(os.path.join(rippl_path, 'rippl')):
        sys.path.extend([rippl_path])
        import rippl
        from rippl.user_settings import UserSettings

        settings = UserSettings()
        settings.settings['paths']['rippl'] = rippl_path

        logging.info('Successfully added RIPPL to path!')
    else:
        logging.info('Wrong RIPPL path, package cannot be imported. Try again:')
except:
    logging.info('Wrong RIPPL path, package cannot be imported. Try again:')

INFO:root:Successfully added RIPPL to path!


As a first step you will have to create an account for the sentinel hub,
which can be done using the following link:

https://dataspace.copernicus.eu/

Running the next block will ask you for your username and password and checks whether these are valid.

(Note: sometimes the server is offline and the program will not be able to verify your account. In that case you can skip this step. In the next step, you will setup a NASA account, which enables you the use of a mirror download site. But be aware: your ESA hub account will not be validated yet. You can do this later by manually adding your username and password to the user_settings.json file, which will be generated later on in this notebook.)

In [2]:
# Initialize settings
from rippl.user_settings import UserSettings
settings = UserSettings()

# Add ESA settings and check whether they are valid.
username = input('Username Copernicus scihub:')
password = input('Password Copernicus scihub:')
success = settings.add_copernicus_settings(username, password)

if success:
    logging.info('Sentinel-1 scihub account added!')

INFO:root:Scihub password valid!
INFO:root:Sentinel-1 scihub account added!


Now create an account for Earthdata which is used to download SRTM DEM data with 1 and 3 arc-second resolution.
This account will also give you access to Sentinel-1 mirror server at the Alaska Satellite Facility, which can be used as an backup for the Copernicus Sentinel hub.

You can create your account using this link:
https://urs.earthdata.nasa.gov//users/new

Now go to the ASF website:
https://search.asf.alaska.edu

If you try to login it will ask you to add some credentials to your account and agree with the license. This makes your account complete.

Running the next block will ask for your username and password again.

In [3]:
# Add Earthdata settings and check whether they are valid.
username = input('Username Earthdata account:')
password = input('Password Earthdata account:')
success = settings.add_earthdata_settings(username, password)

if success:
    logging.info('Earthdata account added!')

INFO:root:Earthdata password valid!
INFO:root:Earthdata account added!


Create an account to download data from the DLR TanDEM-X archive. This step is optional and only
needed when you work with regions above 60 degrees North or 60 degrees South of the equator, as these
areas do not have SRTM coverage. 

There are two different datasets that also require two different accounts. 

For the 30-meter DEM (TDM30) create an account via this link:
https://sso.eoc.dlr.de/tdm30-edited/selfservice/public/newuser
For the 90-meter DEM (TDM90) create an account via this link:
https://sso.eoc.dlr.de/tdm90/selfservice/public/newuser

Running the next block will check you password for the DLR website.

In [4]:
for dem_type in ['TDM30', 'TDM90']:
    logging.info('Username EOC Geoservice (DLR) TanDEM-X DEM account for' + dem_type + ':')
    DLR_username = input()
    logging.info('Username EOC Geoservice (DLR) TanDEM-X DEM account for' + dem_type + ':')
    DLR_password = input()

    success = settings.add_DLR_settings(DLR_username, DLR_password, dem_type)
    if success:
        logging.info('EOC Geoservice (DLR) TanDEM-X DEM account for ' + dem_type + ' added!')


INFO:root:DLR password valid!
INFO:root:DLR TanDEM-X DEM account added!


After the dowload options for SAR data and DEMs we also add two options to add accounts to download NWP model data. One of them is from ERA5 (worldwide) and the other from the KNMI (Netherlands or Western Europe only). You can create an account for the ERA5 data using the following links:

ERA5: https://cds.climate.copernicus.eu/user/register
The requested api key can then be found at:
https://cds.climate.copernicus.eu/ and clicking on your account name in the upper right corner. The UID and API key then given at the bottom of the page.
Downloads will be done using the cds api. To install the python package check out:
https://cds.climate.copernicus.eu/api-how-to

If you are not interested in using NWP model data you kan skip this step
 

In [6]:

api_key = input('API Key for Climate Data Store:')
success = settings.add_ecmwf_settings(api_key)

if success:
    logging.info('CDS account added!')


AttributeError: 'UserSettings' object has no attribute 'add_knmi_settings'

You can create an account for the KNMI data using the following links:

KNMI: https://developer.dataplatform.knmi.nl/register/
The API key can then be found under:
https://developer.dataplatform.knmi.nl/member/

If you are not interested in using NWP model data you kan skip this step

In [3]:
api_key = input('API Key for KNMI:')
success = settings.add_knmi_settings(api_key)

if success:
    logging.info('KNMI account added!') 
    

INFO:root:KNMI account added!


Now we have the needed accounts, we can download the needed data, but you will have to define where to store the data

To do so you will need to create a folder to store:
1. A folder to store the downloaded SAR data. 
2. A folder to store the downloaded DEM data.
3. A folder to store the orbit files. These files are used to determine the exact location at satellite overpass and 
is needed to apply a correct geolocation on the ground. 
4. A folder to write the data_stacks you process. 

Or:

1. Define one primary folder where all other folders will be created automatically. 

Be sure that you have around 50 GB of disk space free to do the processing!

In [2]:
settings.define_sensor_names()

# Specify the main folder only. Leave this one empty if you define the other folders seperately!
main_folder = input('Specify folder for rippl processing. (Leave empty if you want to specify the different data folder separately)')

# Specify the 4 folders seperately
if len(main_folder) == 0:
    radar_database =  input('Specify folder downloaded SAR SLC data')
    DEM_database = input('Specify folder downloaded DEM data')
    orbit_database = input('Specify folder downloaded orbit data')
    radar_data_stacks = input('Specify folder where processed SAR data is stored (preferably on SSD volume for read/write speed)')
    NWP_model_database = input('Specify folder for downloaded NWP data')
    radar_data_products = input('Specify folder for output geotiff files')

    success = settings.save_data_database(radar_database=radar_database,
                                radar_data_stacks=radar_data_stacks,
                                orbit_database=orbit_database,
                                DEM_database=DEM_database,
                                NWP_model_database=NWP_model_database,
                                radar_data_products=radar_data_products)
else:
    success = settings.save_data_database(main_folder=main_folder)
# Save folders

if success:
    logging.info('Folders for processing are set!')

INFO:root:Folders for processing are set!


Before we can download DEM data we will need to index the SRTM database and download the EGM96 geoid model. This is done in the next step. Indexing can take a few minutes... In the newest version this index file comes with the code, so it is not needed to do the indexing here. However, with an update of the download server a rerun of this step could be needed.

In [5]:
# Start by copying the geoid file
import shutil
from rippl.external_dems.geoid import GeoidInterp

egm_96_file=os.path.join(settings.settings['paths']['DEM_database'], 'geoid', 'egm96.dat')
egm_96_file_rippl=os.path.join(rippl_path, 'rippl', 'external_dems', 'egm96.dat')
shutil.copyfile(egm_96_file_rippl, egm_96_file)

egm = GeoidInterp.create_geoid(egm_96_file=egm_96_file)

INFO:root:Succesfully downloaded the EGM96 geoid file to /mnt/external/rippl_tutorial_test/DEM_database/geoid/egm96.dat


To finish the setup of RIPPL install the snaphu software, which is used to do unwrapping. If you are not planning on
doing any unwrapping this is not needed.

On a linux machine this can be done by using the following command:
apt-get install snaphu (add sudo in front if needed)

On a windows machine you could use the prebuild version that is used in the STEP software from ESA. Follow this link:
http://step.esa.int/main/third-party-plugins-2/snaphu/

On a macbook you will have to build the program yourself. You can use the post under the following linke as a reference:
https://forum.step.esa.int/t/installing-snaphu-on-macbook/10969

The source code can be found under:
https://web.stanford.edu/group/radar/softwareandlinks/sw/snaphu/

Make sure that the program is in your system path!

In [8]:
snaphu_path = input('Snaphu path (not needed if you do not want to do unwrapping):')
settings.add_snaphu_path(snaphu_path=snaphu_path)

INFO:root:Snaphu path found and added to RIPPL installation


True

In [9]:
# Save all settings to disk.
settings.save_settings()

logging.info('User settings saved and validated!')

INFO:root:Settings saved to /home/gert/Software/rippl_main/rippl/user_settings.json
INFO:root:User settings saved and validated!
